In [ ]:
import sys
from copy import deepcopy
import time

sys.path.append('../utils')
from pyutils import *

In [ ]:
sample = readutf8('sample.txt')
sample_long = readutf8('sample2.txt')
puzzle = readutf8('input.txt')

test1 = """
#######
#.....#
#.....#
#..O..#
#...O@#
#..O..#
#...O.#
#..O#.#
#...O.#
#.....#
#.....#
#######

<^^<<<vv
"""

In [ ]:
def parse_input(puzzle_in: str, widen: bool=False) -> tuple[StrMatrix, list[Pt], Pt]:
    warehouse_map: StrMatrix = []
    robot_instructions: list[Pt] = []
    robot_pos: Pt = Pt(-1, -1)
    char_to_point = {'^': (-1, 0), '>': (0, 1), 'v': (1, 0), '<': (0, -1)}
    part: int = 1
    for line in puzzle_in.split('\n'):
        if line.strip() == '':
            if len(warehouse_map) > 0:
                part += 1
            continue
        if part == 1:
            if widen:
                wide_line = []
                for ch in line:
                    match ch:
                        case '#': wide_line.extend('##')
                        case 'O': wide_line.extend('[]')
                        case '.': wide_line.extend('..')
                        case '@': wide_line.extend('@.')
                warehouse_map.append(wide_line)
                if '@' in wide_line:
                    robot_pos = Pt(len(warehouse_map) - 1, wide_line.index('@'))
            else:
                warehouse_map.append(line)
                if '@' in line:
                    robot_pos = Pt(len(warehouse_map) - 1, line.index('@'))
        elif part == 2:
            robot_instructions.extend(list(map(lambda i: Pt.of(char_to_point[i]), line)))
    if robot_pos == (-1, -1):
        raise ValueError('Robot position not found in warehouse map!')
    return (warehouse_map, robot_instructions, robot_pos)

In [ ]:
def predict_robot(mat: StrMatrix, instructions: list[Pt], robot_start: Pt):
    box_move_queue: list[tuple[tuple[Pt, Pt], tuple[Pt, Pt]]] = []
    mat = deepcopy(mat)
    def _check_boxes(box_pair: tuple[Pt, Pt]) -> bool:
        box_future = (box_pair[0] + dirpt, box_pair[1] + dirpt)
        ret = True
        # future of [
        if box_future[0] == box_pair[1]:
            ret = True
        elif box_future[0] in checked:
            ret = checked[box_future[0]]
        else:
            match (obj_a := matget(mat, box_future[0])):
                case '#': ret = False
                case '.': ret = True
                case '[' | ']':
                    other_pair = (box_future[0], box_future[1]) if obj_a == '[' else (box_future[0] + (0, -1), box_future[0])
                    ret = _check_boxes(other_pair)
                case _: raise ValueError(f'Unexpected character at {box_future[0]!r}: {obj_a!r}')
            checked[box_future[0]] = ret
        if ret == False:
            box_move_queue.clear()
            return ret
        # future of ]
        if box_future[1] == box_pair[0]:
            ret = True
        elif box_future[1] in checked:
            ret = checked[box_future[1]]
        else:
            match (obj_b := matget(mat, box_future[1])):
                case '#': ret = False
                case '.': ret = True
                case '[' | ']':
                    other_pair = (box_future[1], box_future[1] + (0, 1)) if obj_b == '[' else (box_future[0], box_future[1])
                    ret = _check_boxes(other_pair)
                case _: raise ValueError(f'Unexpected character at {box_future[1]!r}: {obj_b!r}')
            checked[box_future[1]] = ret
        if ret == True:
            box_move_queue.append((box_pair, box_future))
        return ret

    robot_pos = robot_start
    # for dirpt in instructions:
    for n, dirpt in enumerate(instructions):
        # time.sleep(0.005)
        # clear_output(wait=True)
        # print(dirpt)
        # print(f'{n:<8} / {len(instructions) - 1:<8}')
        # print(mat_restring(mat))
        checked: dict[Pt, bool] = {}
        robot_future: Pt = robot_pos + dirpt
        match (obj := matget(mat, robot_future)):
            case '#': continue
            case '.':
                matset(mat, robot_pos, '.')
                robot_pos = robot_future
                matset(mat, robot_pos, '@')
            case '[' | ']':
                box_pair = (robot_future, robot_future + (0, 1)) if obj == '[' else (robot_future + (0, -1), robot_future)
                box_move_queue.clear()
                result = _check_boxes(box_pair)
                if result == True:
                    for box_from, _ in box_move_queue:
                        matset(mat, box_from[0], '.')
                        matset(mat, box_from[1], '.')
                    for _, box_to in box_move_queue:
                        matset(mat, box_to[0], '[')
                        matset(mat, box_to[1], ']')
                    matset(mat, robot_pos, '.')
                    robot_pos = robot_future
                    matset(mat, robot_pos, '@')
    return mat

In [ ]:
def box_gps_coords(mat: StrMatrix) -> list[int]:
    coords: list[int] = []
    for pt, i in mat_iter(mat):
        if i == '[':
            coords.append((100 * pt[0]) + pt[1])
    return coords

In [ ]:
warehouse, robot_moves, robot = parse_input(puzzle, widen=True)
# print(mat_restring(warehouse))

In [ ]:
# predict_robot(warehouse, robot_moves, robot)

In [ ]:
print(sum(box_gps_coords(predict_robot(warehouse, robot_moves, robot))))